# 제품 이상여부 판별 프로젝트


## 데이터 불러오기


### 필수 라이브러리


In [68]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
from pprint import pprint

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기


In [69]:
import pandas as pd

THRESHOLD = 0.3
RANDOM_STATE = 110

train_data = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/train_data_0816.csv")
test_data = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/test_data_0816.csv")

In [70]:
# 공통 변수 리스트
com_variables_train = [
    'target', 'Model.Suffix', 'Workorder', 'WorkMode Collect Result'
    , 'Dispenser_1', 'Dispenser_2', 'Receip_No_Collect_Result'
    , 'Production_Qty_Collect_Result', 'Judge_Value_OK'
    , 'Workorder_0.9', 'Workorder_0.6'
]

com_variables_test = [
    'target', 'Set ID', 'Model.Suffix', 'Workorder'
    , 'WorkMode Collect Result', 'Dispenser_1'
    , 'Dispenser_2', 'Receip_No_Collect_Result'
    , 'Production_Qty_Collect_Result', 'Judge_Value_OK'
    , 'Workorder_0.9', 'Workorder_0.6'
]

In [71]:
# 공정 이름 필터링 후 공통 변수와 결합
def create_dataset(train_data, test_data, process_name, com_variables_train, com_variables_test):
    # 열 이름 필터링
    Process_Desc_col = train_data.filter(like=process_name).columns
    
    # train 데이터셋 생성
    final_columns_train = list(Process_Desc_col) + com_variables_train
    train_dataset = train_data[final_columns_train]
    
    # test 데이터셋 생성
    final_columns_test = list(Process_Desc_col) + com_variables_test
    test_dataset = test_data[final_columns_test]
    
    return train_dataset, test_dataset

# 공통 변수 정의
## com_variables_train = [...]  -> 이전 코드에서 정의한 변수 사용
## com_variables_test = [...]   -> 이전 코드에서 정의한 변수 사용

# 데이터셋 생성
train_data_dam, test_data_dam = create_dataset(train_data, test_data, '_Dam', com_variables_train, com_variables_test)
train_data_fill1, test_data_fill1 = create_dataset(train_data, test_data, '_Fill1', com_variables_train, com_variables_test)
train_data_fill2, test_data_fill2 = create_dataset(train_data, test_data, '_Fill2', com_variables_train, com_variables_test)
train_data_autoclave, test_data_autoclave = create_dataset(train_data, test_data, '_AutoClave', com_variables_train, com_variables_test)

---

## 모델링

### 모델 정의

In [72]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score

# 스레드홀드 설정
THRESHOLD = 0.3

# 모델 설정 및 하이퍼파라미터
models = {
    'et': ExtraTreesClassifier(),
    'rf': RandomForestClassifier(),
    'cat': CatBoostClassifier(),
    'lgbm': LGBMClassifier(),
    'xgb': XGBClassifier(),
    'dt': DecisionTreeClassifier()
}

def train_and_evaluate_model(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return
    
    # 데이터셋 분할
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # 모델 선택
    model = models[model_name]

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    # 예측
    y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'{model_name} 모델이 {data_name} 데이터로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

# 사용 예시
# train_and_evaluate_model(
#     'lgbm', train_data_fill1,
#     n_estimators=979,
#     num_leaves=1565,
#     max_depth=34,
#     learning_rate=0.04888906225539191,
#     min_child_samples=36,
#     boosting_type='dart',
#     random_state=RANDOM_STATE,
#     verbose=-1
# )


def fit_all_train_data_function(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return None  # 지원되지 않는 모델일 경우 None 반환
    
    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(data.drop("target", axis=1), data["target"].map({'Normal': 0, 'AbNormal': 1}))

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    print(f'{model_name} 모델이 {data_name} 데이터로 학습 완료')
    return model  # 학습된 모델 반환

# 사용 예시
# model_dam = fit_all_train_data_function(
#     'lgbm', train_data_dam,
#     n_estimators=2748,
#     num_leaves=657,
#     max_depth=256,
#     learning_rate=0.001043279508273329,
#     min_child_samples=58,
#     boosting_type='dart',
#     random_state=RANDOM_STATE,
#     verbose=-1
# )


### 모델 학습

Dam 모델

In [73]:
train_and_evaluate_model(
    'rf', train_data_dam
    , n_estimators = 2265
    , max_depth = 18
    , min_samples_split = 11
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
    , random_state=RANDOM_STATE
)

rf 모델이 train_data_dam 데이터로 학습한 결과:
F1 Score: 0.2263513513513514
---
Confusion Matrix:
[[7577   85]
 [ 373   67]]
---
Accuracy: 0.9434707479634659
Precision: 0.4407894736842105
Recall: 0.15227272727272728




AutoClave 모델

In [74]:
train_and_evaluate_model(
    'rf', train_data_autoclave
    , n_estimators = 2265
    , max_depth = 18
    , min_samples_split = 11
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
    , random_state=RANDOM_STATE
)

rf 모델이 train_data_autoclave 데이터로 학습한 결과:
F1 Score: 0.22641509433962265
---
Confusion Matrix:
[[7585   77]
 [ 374   66]]
---
Accuracy: 0.9443347321648976
Precision: 0.46153846153846156
Recall: 0.15




Fill1 모델

In [75]:
train_and_evaluate_model(
    'rf', train_data_fill1
    , n_estimators = 2265
    , max_depth = 18
    , min_samples_split = 11
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
    , random_state=RANDOM_STATE
)

rf 모델이 train_data_fill1 데이터로 학습한 결과:
F1 Score: 0.22553897180762847
---
Confusion Matrix:
[[7567   95]
 [ 372   68]]
---
Accuracy: 0.9423599111330536
Precision: 0.4171779141104294
Recall: 0.15454545454545454




Fill2 모델

In [76]:
train_and_evaluate_model(
    'rf', train_data_fill2
    , n_estimators = 2265
    , max_depth = 18
    , min_samples_split = 11
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
    , random_state=RANDOM_STATE
)

rf 모델이 train_data_fill2 데이터로 학습한 결과:
F1 Score: 0.21069182389937108
---
Confusion Matrix:
[[7533  129]
 [ 373   67]]
---
Accuracy: 0.9380399901258949
Precision: 0.34183673469387754
Recall: 0.15227272727272728




---

### 모델 학습(train 데이터 전체 학습)

위의 모델학습 코드에서 함수명만 바뀌고 들어가는 값들은 동일  
-> 위의 코드 복붙한다음 함수명만 바꿔주면 사용하기 편함  

In [77]:
model_Dam = fit_all_train_data_function(
    'rf', train_data_dam
    , n_estimators = 2265
    , max_depth = 18
    , min_samples_split = 11
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
)

model_AutoClave = fit_all_train_data_function(
    'rf', train_data_autoclave
    , n_estimators = 2265
    , max_depth = 18
    , min_samples_split = 11
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
)

model_Fill1 = fit_all_train_data_function(
    'rf', train_data_fill1
    , n_estimators = 2265
    , max_depth = 18
    , min_samples_split = 11
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
)

model_Fill2 = fit_all_train_data_function(
    'rf', train_data_fill2
    , n_estimators = 2265
    , max_depth = 18
    , min_samples_split = 11
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
)

rf 모델이 train_data_dam 데이터로 학습 완료
rf 모델이 train_data_autoclave 데이터로 학습 완료
rf 모델이 train_data_fill1 데이터로 학습 완료
rf 모델이 train_data_fill2 데이터로 학습 완료


---

In [78]:
# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)

# 각 공정의 예측 확률 계산
probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
]

In [79]:
# 소프트 보팅: 각 모델의 확률 평균 계산
soft_voting_probs = np.mean(probs, axis=0)

# 최종 예측: 평균 확률에 대해 스레드 홀드 0.3 적용
final_predictions = (soft_voting_probs >= 0.3).astype(int)

# 최종 예측 결과 출력
print(sum(final_predictions))

218


In [88]:
# 소프트 보팅: 각 모델의 확률 평균 계산
soft_voting_probs = np.mean(probs, axis=0)

# 최종 예측: 평균 확률에 대해 스레드 홀드 0.28 적용
final_predictions = (soft_voting_probs >= 0.2).astype(int)

# 최종 예측 결과 출력
print(sum(final_predictions))

584


## 4. 제출하기


### 제출 파일 작성


In [91]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

In [90]:
df_sub['target'].value_counts()

target
Normal      16777
AbNormal      584
Name: count, dtype: int64

In [83]:
df_sub.head(10)

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,Normal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,Normal
3,000a6bfd02874c6296dc7b2e9c5678a7,AbNormal
4,0018e78ce91343678716e2ea27a51c95,Normal
5,001fda4596f545d0a3b0ce85fbea77d2,Normal
6,0020734a7b29472298358ad58645a0c9,Normal
7,00234c5914cd4c4a888d13f8b3773135,Normal
8,00297b6c93e44d49ac534758a23dc74e,Normal
9,002d904240d84b188d410d16383a9c3a,Normal


**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**

Public Score : 0.22222222222222224
